In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [70]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_sub = pd.read_csv('sample_submission.csv')

In [71]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             371 non-null    int64  
 1   bone_length    371 non-null    float64
 2   rotting_flesh  371 non-null    float64
 3   hair_length    371 non-null    float64
 4   has_soul       371 non-null    float64
 5   color          371 non-null    object 
 6   type           371 non-null    object 
dtypes: float64(4), int64(1), object(2)
memory usage: 20.4+ KB


In [72]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 529 entries, 0 to 528
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             529 non-null    int64  
 1   bone_length    529 non-null    float64
 2   rotting_flesh  529 non-null    float64
 3   hair_length    529 non-null    float64
 4   has_soul       529 non-null    float64
 5   color          529 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 24.9+ KB


In [73]:
train_df.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [74]:
train_df.groupby('type')['bone_length'].mean()

type
Ghost     0.347466
Ghoul     0.517159
Goblin    0.429650
Name: bone_length, dtype: float64

In [75]:
train_df.groupby('type')['rotting_flesh'].mean()

type
Ghost     0.593759
Ghoul     0.490515
Goblin    0.442357
Name: rotting_flesh, dtype: float64

In [76]:
train_df.groupby('type')['hair_length'].mean()

type
Ghost     0.373186
Ghoul     0.655550
Goblin    0.544581
Name: hair_length, dtype: float64

In [77]:
train_df.groupby('type')['has_soul'].mean()

type
Ghost     0.320023
Ghoul     0.601634
Goblin    0.478663
Name: has_soul, dtype: float64

In [78]:
full_df = pd.concat([train_df, test_df], axis = 0, ignore_index = True)

In [79]:
full_df

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost
...,...,...,...,...,...,...,...
895,893,0.377573,0.390158,0.696465,0.355373,blue,NaN
896,894,0.229161,0.601265,0.191282,0.475115,clear,NaN
897,895,0.510497,0.498347,0.708020,0.714154,white,NaN
898,898,0.331472,0.765835,0.338207,0.193431,clear,NaN


In [80]:
full_df.color.unique()

array(['clear', 'green', 'black', 'white', 'blue', 'blood'], dtype=object)

In [81]:
color_dummies = pd.get_dummies(full_df['color'], prefix = 'color', drop_first = True)

In [82]:
full_df.drop('color', axis = 1, inplace = True)
full_df = pd.concat([full_df, color_dummies], axis = 1)

In [83]:
full_df

,id,bone_length,rotting_flesh,hair_length,has_soul,type,color_blood,color_blue,color_clear,color_green,color_white
0,0,0.354512,0.350839,0.465761,0.781142,Ghoul,0,0,1,0,0
1,1,0.575560,0.425868,0.531401,0.439899,Goblin,0,0,0,1,0
2,2,0.467875,0.354330,0.811616,0.791225,Ghoul,0,0,0,0,0
3,4,0.776652,0.508723,0.636766,0.884464,Ghoul,0,0,0,0,0
4,5,0.566117,0.875862,0.418594,0.636438,Ghost,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
895,893,0.377573,0.390158,0.696465,0.355373,NaN,0,1,0,0,0
896,894,0.229161,0.601265,0.191282,0.475115,NaN,0,0,1,0,0
897,895,0.510497,0.498347,0.708020,0.714154,NaN,0,0,0,0,1
898,898,0.331472,0.765835,0.338207,0.193431,NaN,0,0,1,0,0


In [84]:
X_train = full_df.loc[:370, full_df.drop(['id','type'], axis = 1).columns]

In [85]:
X_train.tail

<bound method NDFrame.tail of      bone_length  rotting_flesh  hair_length  has_soul  color_blood  \
0       0.354512       0.350839     0.465761  0.781142            0   
1       0.575560       0.425868     0.531401  0.439899            0   
2       0.467875       0.354330     0.811616  0.791225            0   
3       0.776652       0.508723     0.636766  0.884464            0   
4       0.566117       0.875862     0.418594  0.636438            0   
..           ...            ...          ...       ...          ...   
366     0.458132       0.391760     0.660590  0.635689            0   
367     0.331936       0.564836     0.539216  0.551471            0   
368     0.481640       0.501147     0.496446  0.544003            0   
369     0.294943       0.771286     0.583503  0.300618            0   
370     0.670200       0.768469     0.737274  0.608384            0   

     color_blue  color_clear  color_green  color_white  
0             0            1            0            0  
1  

In [86]:
y_train = full_df.type[:371]
y_train.tail

<bound method NDFrame.tail of 0       Ghoul
1      Goblin
2       Ghoul
3       Ghoul
4       Ghost
        ...  
366    Goblin
367     Ghost
368     Ghoul
369     Ghost
370     Ghoul
Name: type, Length: 371, dtype: object>

In [87]:
map_dict = {'Ghost': 0, 'Ghoul': 1, 'Goblin': 2}
y_train = y_train.map(map_dict)

In [88]:
X_test = full_df.loc[371:, full_df.drop(['id','type'], axis = 1).columns]

In [125]:
from sklearn.linear_model import  LogisticRegression
log_r = LogisticRegression()
log_r.fit(X_train, y_train)

LogisticRegression()

In [126]:
y_test = pd.Series(log_r.predict(X_test))

In [127]:
sub = pd.DataFrame({'id': test_df.id.values, 'type': y_test.values})

In [128]:
sub

,id,type
0,3,1
1,6,1
2,9,1
3,10,0
4,13,0
...,...,...
524,893,2
525,894,0
526,895,1
527,898,0


In [129]:
sub.type = sub.type.replace(0, 'Ghost').replace(1, 'Ghoul').replace(2, 'Goblin')

In [130]:
sub

,id,type
0,3,Ghoul
1,6,Ghoul
2,9,Ghoul
3,10,Ghost
4,13,Ghost
...,...,...
524,893,Goblin
525,894,Ghost
526,895,Ghoul
527,898,Ghost


In [131]:
sub.to_csv('sub.csv', index = False)